# LangChain Agent Test

## Outline

 * Direct API calls to OpenAI API (custom model that enabled by `llama-cpp-python` package).
 * To see whether the custom model has agent capability.
 * Using built in LangChain tools: DuckDuckGo search and Wikipedia
 * Defining your own tools

## Setup

In [ ]:
%pip install --no-cache-dir -qU python-dotenv openai langchain wikipedia

In [1]:
import openai
dir(openai)

['APIError',
 'Audio',
 'Callable',
 'ChatCompletion',
 'Completion',
 'ContextVar',
 'Customer',
 'Deployment',
 'Edit',
 'Embedding',
 'Engine',
 'ErrorObject',
 'File',
 'FineTune',
 'Image',
 'InvalidRequestError',
 'Model',
 'Moderation',
 'OpenAIError',
 'Optional',
 'TYPE_CHECKING',
 'Union',
 'VERSION',
 '__all__',
 '__annotations__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'aiosession',
 'api_base',
 'api_key',
 'api_key_path',
 'api_requestor',
 'api_resources',
 'api_type',
 'api_version',
 'app_info',
 'ca_bundle_path',
 'datalib',
 'debug',
 'enable_telemetry',
 'error',
 'log',
 'openai_object',
 'openai_response',
 'organization',
 'os',
 'proxy',
 'requestssession',
 'sys',
 'util',
 'verify_ssl_certs',
 'version']

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

openai.api_key = os.environ['OPENAI_API_KEY']
openai.api_base = os.environ['OPENAI_API_BASE']

In [4]:
#REF: "User: Hello<|end_of_turn|>Assistant: Hi<|end_of_turn|>User: How are you today?<|end_of_turn|>Assistant:"
DEFAULT_SYSTEM_PROMPT = "You are a helpful assistant, you will complete any task by follow instructions given."
EOT = "<|end_of_turn|>"
def get_prompt(system=DEFAULT_SYSTEM_PROMPT, instruction=None):
    if not instruction:
        raise ValueError("instruction must be specified")
    prompt = f"System: {system}{EOT}User: {instruction}{EOT}Assistant:"    
    return prompt

In [5]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": get_prompt(instruction=prompt)}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, 
        max_tokens=1024
    )
    return response.choices[0].message["content"]


In [6]:
get_completion("What is 1+1?")

' 1+1 equals 2.###\n\n'

## Agent Test

In [7]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.chat_models import ChatOpenAI

In [8]:
llm = ChatOpenAI(temperature=0, max_tokens=1024)

In [9]:
tools = load_tools(["llm-math","wikipedia"], llm=llm)

In [10]:
agent= initialize_agent(
    tools, 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose=True)

In [11]:
# agent("What is the 25% of 300?") doesn't works!
%%time
agent("Calculate the value of 25% of 300.")




> Entering new AgentExecutor chain...
Could not parse LLM output: 
{
  "action": Calculator,
  "action_input": 25 / 100 * 300
}


Observation: Invalid or incomplete response
Thought:Final Answer: 75
```
{
  "action": Calculator,
  "action_input": 25 / 100 * 300
}
```



> Finished chain.


{'input': 'Calculate the value of 25% of 300.',
 'output': '75\n```\n{\n  "action": Calculator,\n  "action_input": 25 / 100 * 300\n}\n```'}

In [12]:
%%time
agent("What is the 25% of 300?")



> Entering new AgentExecutor chain...
Could not parse LLM output: Action: Calculator
action_input: 25
```
{
  "action": Calculator,
  "action_input": 25
}
```

Thought: I will calculate 25% of 300.


Observation: Invalid or incomplete response
Thought: The 25% of 300 is 75.
Final Answer: 75

> Finished chain.
CPU times: user 266 ms, sys: 4.44 ms, total: 270 ms
Wall time: 4min 58s


{'input': 'What is the 25% of 300?', 'output': '75'}

In [13]:
%%time
question = "Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU)\
what book did he write?"
result = agent(question)
result



> Entering new AgentExecutor chain...


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIError: Invalid response object from API: '{"description":"Internal Server Error","status":500,"message":"The application encountered an unexpected error and could not continue."}' (HTTP response code was 500).
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIError: Invalid response object from API: '{"description":"Not Found","status":404,"message":"Not Found"}' (HTTP response code was 404).
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIError: Invalid response object from API: '{"description":"Not Found","status":404,"message":"Not Found"}' (HTTP response code was 404).
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 8.0 second

APIError: Invalid response object from API: '{"description":"Not Found","status":404,"message":"Not Found"}' (HTTP response code was 404)

## Python Agent

In [ ]:
agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    verbose=True
)

In [ ]:
customer_list = [["Harrison", "Chase"], 
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"], 
                 ["Geoff","Fusion"], 
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]

In [ ]:
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 

#### View detailed outputs of the chains

In [ ]:
import langchain
langchain.debug=True
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 
langchain.debug=False

## Define your own tool

In [ ]:
#!pip install DateTime

In [ ]:
from langchain.agents import tool
from datetime import date

In [ ]:
@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())

In [ ]:
agent= initialize_agent(
    tools + [time], 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

**Note**: 

The agent will sometimes come to the wrong conclusion (agents are a work in progress!). 

If it does, please try running it again.

In [ ]:
try:
    result = agent("whats the date today?") 
except: 
    print("exception on external access")